# LangChain 核心模块 Agent - Self ask with search

- Google 搜索对接
    - 第三方平台：https://serpapi.com
    - LangChain API 封装：`SerpAPIWrapper`
- LangChain Agent 模块使用
    - Agent 类型：`SELF_ASK_WITH_SEARCH`
    - Agent 实例化：`initialize_agent` 方法
 
https://python.langchain.com/docs/modules/agents/agent_types/

In [1]:
# import os

# 更换为自己的 Serp API KEY
# os.environ["SERPAPI_API_KEY"] = "1e3c93e0753ac224098370cd71da86150c6609caba6b5aaa307e04b72a5006e1"

In [3]:
!pip install google-search-results

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32078 sha256=13ccde94fa0f13255eaa5c820f3e96b7a6b759b6fdcba24f37260e7b403a551b
  Stored in directory: c:\users\huxiuyu\appdata\local\pip\cache\wheels\d3\b2\c3\03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [17]:
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, AgentType, Tool

llm = OpenAI(temperature=0)

In [18]:
# 实例化查询工具
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

In [19]:
# 实例化 SELF_ASK_WITH_SEARCH Agent
self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

In [9]:
# 实际运行 Agent，查询问题（正确）
self_ask_with_search.run(
    "成都举办的大运会是第几届大运会？"
)



> Entering new AgentExecutor chain...
 Yes.
Follow up: 什么时候举办的成都大运会？
Intermediate answer: Fri, Jul 28, 2023, 8:00 AM – Tue, Aug 8, 2023
Follow up: 成都大运会是第几届大运会？
Intermediate answer: 2021年夏季世界大学生运动会（英语：2021 Summer World University Games）简称为2021年成都大运会或成都大运会，是第31届夏季世界大学生运动会，原定于2021年于中国四川省成都市举行，后因2019冠状病毒病疫情等原因二次延期，最终改至2023年7月28日至8月8日举行。
So the final answer is: 第31届

> Finished chain.


'第31届'

In [8]:
print(llm("成都举办的大运会是第几届大运会？"))



成都举办的是第十一届大运会。


In [21]:
# 实际运行 Agent，查询问题（有歧义）
self_ask_with_search.run(
    "2023年夏季大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...
 Yes.
Follow up: What is the name of the city hosting the 2023 Summer Olympics?
Intermediate answer: In 2023, a slew of Paris 2024 Olympic Games-related events will take place all over the world. From qualifiers to test events, ticketing phases and medal design reveals, here are all the things you won't want to miss in 2023.
Follow up: Where is the main venue for the 2023 Summer Olympics?
Intermediate answer: In 2023, a slew of Paris 2024 Olympic Games-related events will take place all over the world.
The main venue for the 2023 Summer Olympics will be in Paris, France.
So the final answer is: Paris, France.

> Finished chain.


'Paris, France.'

In [12]:
# Reason-only 答案（错误）
print(llm("2023年大运会举办地在哪里？"))



2023年大运会将在日本东京举办。


#### 使用 GPT-4 作为大语言模型实现更优的 ReAct 范式

In [13]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4", temperature=0)

In [14]:
self_ask_with_search_chat = initialize_agent(
    tools, chat_model, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

In [16]:
# GPT-4 based ReAct 答案（正确）
self_ask_with_search_chat.run(
    "2023年夏季大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...
Yes.
Follow up: 2023年夏季大运会是哪个国际赛事的名称？
Intermediate answer: 国际大学生体育联合会世界大学生运动会（英语：FISU World University Games），简称世界大学生运动会、大学生运动会、大运会，是由国际大学生体育联合会（法语：Fédération Internationale du Sport Universitaire，缩写FISU）主办，专供大学生运动员参加的 ...
Follow up: 2023年的世界大学生运动会举办地在哪里？
Intermediate answer: 成都第31届世界大学生夏季运动会，将于7月28日开幕。 第31届世界大学生夏季运动会将于7月28日至8月8日在四川成都举行。 在龙泉驿区的东安湖畔，形如“银色飞碟”的东安湖体育公园主体育场，是大运会开幕式主场馆，成为蓉城又一地标。
So the final answer is: 成都

> Finished chain.


'成都'